In [ ]:
using Pkg; Pkg.activate("../../");
using BenchmarkTools
using Dates, Fastback

In [ ]:
# synthetic data
N = 100_000
prices = 1000.0 .+ cumsum(randn(N) .+ 0.01)
bids = prices .- 0.01
asks = prices .+ 0.01
dts = map(x -> DateTime(2000, 1, 1) + Minute(x) + Millisecond(123), 1:N);

In [ ]:
# create instrument
inst = Instrument(1, "AAPL")
insts = [inst]

# market data (order books)
data = MarketData(insts)

# create trading account
acc = Account(insts, 100_000.0)

# plot data collectors
collect_balance, balance_curve = periodic_collector(Float64, Second(1))
collect_equity, equity_curve = periodic_collector(Float64, Second(1))
collect_open_orders, open_orders_curve = max_value_collector(Int64)
collect_drawdown, drawdown_curve = drawdown_collector(Percentage, (v, dt, equity) -> dt - v.last_dt >= Second(1))

pos = acc.positions[inst.index]

# backtest random trading strategy
for i in 1:N
    dt = dts[i]
    book = data.order_books[inst.index]
    update_book!(book, BidAsk(dts[i], bids[i], asks[i]))

    if i == N
        # close all orders at end of backtest
        if pos.quantity !== 0.0
            execute_order!(acc, book, Order(inst, -pos.quantity, dt))
        end
    else
        # randomly trade
        if rand() > 0.999
            sgn = rand() >= 0.5 ? 1.0 : -1.0
            execute_order!(acc, book, Order(inst, sgn, dt))
            # println("Net exposure: $(pos.quantity)")
        end

        # # close positions after 10 minutes
        # if is_exposed_to(acc, inst)
        #     if dt - pos.orders_history[end].dt >= Minute(10)
        #         execute_order!(acc, book, Order(inst, -pos.quantity, dt))
        #     end
        # end
    end

    update_account!(acc, data, inst)

    # collect data for plotting
    collect_balance(dt, acc.balance)
    collect_equity(dt, acc.equity)
    collect_open_orders(dt, length(acc.positions))
    collect_drawdown(dt, acc.equity)
end

# print account
show(acc; max_orders=1000)

println("Sum realized pnl: $(sum(o.execution.realized_pnl for o in acc.orders_history))")
println("Net account pnl:  $(acc.equity - acc.initial_balance)")

In [ ]:
function run(dts, bids, asks)
    N = length(dts)
    
    # create instrument
    inst = Instrument(1, "AAPL")
    insts = [inst]

    # market data (order books)
    data = MarketData(insts)

    # create trading account
    acc = Account(insts, 100_000.0)

    pos = acc.positions[inst.index]

    # backtest random trading strategy
    for i in 1:N
        dt = dts[i]
        book = data.order_books[inst.index]
        update_book!(book, BidAsk(dts[i], bids[i], asks[i]))

        if i == N
            # close all orders at end of backtest
            if pos.quantity !== 0.0
                execute_order!(acc, book, Order(inst, -pos.quantity, dt))
            end
        else
            # randomly trade
            if rand() > 0.999
                sgn = rand() >= 0.5 ? 1.0 : -1.0
                execute_order!(acc, book, Order(inst, sgn, dt))
            end
        end

        update_account!(acc, data, inst)
    end
end

In [ ]:
@benchmark run(dts, bids, asks) samples=30 evals=3

In [ ]:
@code_warntype run(dts, bids, asks)

In [ ]:
# using ProfileView
# ProfileView.@profview map(i -> run(dts, bids, asks), 1:10)